# Summary

Below, we take a small table and go through the process of aligning and projecting the galaxies.</br>
We assume we are given a galaxy table with 3D information. However, if we only have RA, dec, and redshift, as well as the cosmology used,</br>
there is a function in modular_alignment.py to get cartesian coordinates from that.</br></br>

This process is also shown in the inject_ia_to_lightcone function in modular_alignmnet.py, but hopefully this representation</br>
is clearer and easier to follow.

# Import and Setup

In [1]:
from modular_alignment import align_to_halo, align_radially, get_galaxy_positions, project_alignments_with_NCP, get_position_angle
from astropy.io import ascii
import numpy as np

In [2]:
# This dict is just a mapping between what I call the value and the key used in the astropy table
# Maybe a slight bit hacky, but it lets me use a consistent naming scheme as long as the user provides some similar dict

table_keys = {
    "ra" : "ra_true",                                                   # RA
    "dec" : "dec_true",                                                 # dec
    "redshift" : "redshiftHubble",                                      # Redshift
    "x" : "x",
    "y" : "y",
    "z" : "z",
    "halo_x" : "baseDC2/target_halo_x",
    "halo_y" : "baseDC2/target_halo_y",
    "halo_z" : "baseDC2/target_halo_z",
    "halo_axisA_x" : "baseDC2/target_halo_axis_A_x",
    "halo_axisA_y" : "baseDC2/target_halo_axis_A_y",
    "halo_axisA_z" : "baseDC2/target_halo_axis_A_z",
    "halo_mvir" : "baseDC2/host_halo_mvir",
    "isCentral" : "isCentral",
    "ellipticity" : "morphology/totalEllipticity",
    "e1" : "morphology/totalEllipticity1",
    "e2" : "morphology/totalEllipticity2",
    "gamma1" : "shear1",
    "gamma2" : "shear2",
    "g" : "mag_true_g",
    "r" : "mag_true_r",
    "stellar_mass" : "stellar_mass",
    "s11" : "tidal_s_11",
    "s12" : "tidal_s_12",
    "s22" : "tidal_s_22"
}

Read in a short example table.</br>
One central galaxy and two satellites (not necessarily related)

In [3]:
data = ascii.read("short_table.dat")
data["isCentral"] = data["isCentral"] == "True"

In [4]:
data

ra_true,dec_true,redshiftHubble,x,y,z,baseDC2/target_halo_x,baseDC2/target_halo_y,baseDC2/target_halo_z,baseDC2/target_halo_axis_A_x,baseDC2/target_halo_axis_A_y,baseDC2/target_halo_axis_A_z,baseDC2/host_halo_mvir,baseDC2/target_halo_mass,isCentral,morphology/totalEllipticity,morphology/totalEllipticity1,morphology/totalEllipticity2,shear1,shear2,mag_true_g,mag_true_r,stellar_mass,tidal_s_11,tidal_s_12,tidal_s_22
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.6554793009305763,-40.187277441739155,0.20298659801483154,445.3499755859375,5.095147132873535,-376.2050476074219,445.3499755859375,5.095147132873535,-376.2050476074219,-0.06459414886164111,-0.054491909701089906,0.12305640417054158,126449999872.0,158923784192.0,True,0.5069335103034973,-0.4976382255554199,0.0966322124004364,-0.0010780795044078029,-0.0001935919043020633,22.277347564697266,21.623077392578125,475602400.0,-0.2818889944672113,0.40005400124426527,0.38830382637543054
0.702982389058322,-39.47357707949621,0.20586907863616943,456.1093484190837,5.596449378418383,-375.6626645289206,456.0932922363281,5.665058135986328,-375.6200256347656,0.3174143488982946,-0.051942194453338246,0.2659174734755303,2824100118528.0,3472242507776.0,False,0.03980273753404617,-0.01758822798728943,0.03570591285824776,4.247972215298823e-05,-0.001404673295386511,22.53379249572754,21.499557495117188,1453857664.0,0.2174605605253552,0.36815189641080087,0.8146693157568672
0.7200102146955414,-39.482723243105056,0.20586907863616943,456.02346422424995,5.730942836655683,-375.71550924438276,456.0932922363281,5.665058135986328,-375.6200256347656,0.3174143488982946,-0.051942194453338246,0.2659174734755303,2824100118528.0,3472242507776.0,False,0.45927709341049194,0.20627683401107788,-0.4103478193283081,-2.3903166125993636e-05,-0.0009506534437888547,21.415847778320312,20.174400329589844,7387039232.0,-0.0837378781577978,0.4144596483241478,0.6740889367478052


# Align

## Get the right variables

### From the data table
Here, I am grabbing them from an existing galaxy catalog. However, as long as you get these values (regardless of how), that's all the next step needs.

In [5]:
# First, get the mask for central galaxies
central_mask = data["isCentral"]

# Second, get the values from the appropriate columns
# Here's where that weird dict above comes in handy

# Major axes
halo_axisA_x = data[ table_keys["halo_axisA_x"] ]                # Grab the column with whatever the catalog's version of "halo_axisA_x" is called
halo_axisA_y = data[ table_keys["halo_axisA_y"] ]
halo_axisA_z = data[ table_keys["halo_axisA_z"] ]

# Central/Parent halo positions
halo_x = data[ table_keys["halo_x"] ]
halo_y = data[ table_keys["halo_y"] ]
halo_z = data[ table_keys["halo_z"] ]

# Galaxy positions
x = data[ table_keys["x"] ]
y = data[ table_keys["y"] ]
z = data[ table_keys["z"] ]

### Extra variables
We also need alignment strengths and the box size

In [6]:
central_alignment_strength = 0.8
satellite_alignment_strength=0.5

Lbox = np.array([500., 500., 500.])

In [17]:
halo_axisA_x[central_mask]

-0.06459414886164111


## The Actual Alignment
Perform the alignment and get the galaxy major axis orientation.</br>
We also get the intermediate and minor axes, but these are just random perpendicular vectors

In [7]:
# Don't forget to use the proper mask
# This way, we are only passing in the values needed for aligning the centrals or satellites
cen_major, cen_inter, cen_minor = align_to_halo( halo_axisA_x[central_mask], halo_axisA_y[central_mask], halo_axisA_z[central_mask],
                                                 central_alignment_strength, prim_gal_axis="A" )

sat_major, sat_inter, sat_minor = align_radially( halo_x[~central_mask], halo_y[~central_mask], halo_z[~central_mask], 
                                                     x[~central_mask], y[~central_mask], z[~central_mask], 
                                                     Lbox, satellite_alignment_strength, prim_gal_axis="A" )

In [20]:
cen_major

array([[-0.0046144 ,  0.49676933, -0.86787035]])

## Project

In [37]:
# First, get the positions (just split up what we grabbed earlier)
# We want these as a 2D array. First column has x, second column has y, third column has z
cen_coords = np.array( [x[central_mask], y[central_mask], z[central_mask]] ).T
sat_coords = np.array( [x[~central_mask], y[~central_mask], z[~central_mask]] ).T

# Now project onto the plane perpendicular to the line of sight
# Along with this projection, we will the get vertical and horizontal axes of that plane
# where vertical is the projection of the North Celestial Pole (NCP) and the horizontal is West
cen_projected_axes, cen_north, cen_west = project_alignments_with_NCP( cen_major, cen_coords )
sat_projected_axes, sat_north, sat_west = project_alignments_with_NCP( sat_major, sat_coords )

In [38]:
cen_projected_axes

array([[-0.49678961, -0.66231051]])

## Get Position Angles
Finally, now that we have projected, our 2D orientation can be represented simply as the position angle taken counterclockwise with</br>
respect to the NCP

In [39]:
cen_phi = get_position_angle(cen_projected_axes)
sat_phi = get_position_angle(sat_projected_axes)

With these angles, we have our 2D alignment. If youhave a list of galaxy ellipticities, you can get e1 and e2 using that and this position angle.

In [40]:
cen_phi

array([2.49803673])

In [31]:
cen_projected_axes

array([[0., 0.],
       [0., 0.],
       [0., 0.]])